In [ ]:
!pip install pacmap
import pacmap
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

# loading dataset

#read the features
x = read_csv('X1.csv')

X = x.values
X = X.transpose()
print(X.shape)

#read the labels
y = read_csv('Y1.csv')

Y = y.values
# defining and initiliazing the PacMap
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)

# fit the data (The index of transformed data corresponds to the index of the original data)
X_transformed = embedding.fit_transform(X, init="pca")
print(X_transformed)
# visualize the embedding
#fig, ax = plt.subplots(1, 1, figsize=(6, 6))
#ax.scatter(X_transformed[:, 0], X_transformed[:, 1], cmap="Spectral", c=y, s=0.6)


In [ ]:
# The part of the code has been taken from https://github.com/NaziaFatima/iSOM_GSN
# -*- coding: utf-8 -*-
"""
Created on Mon May  6 15:20:11 2019
"""

if __name__ == "__main__":
#    run_colorsExample()
    import pandas as pd
    import networkx as nx
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import MinMaxScaler
    import os, sys

#    pos = [[9.5, 7.794228634059948],  [0, 5.196152422706632],  [9.5, 2.598076211353316],  [9, 5.196152422706632],  [6.5, 2.598076211353316],  [4.5, 7.794228634059948],  [0, 3.4641016151377553],  [9, 0.0],  [2, 6.9282032302755105],  [5.5, 7.794228634059948],  [5, 0.0],  [4, 5.196152422706632],  [0, 6.9282032302755105],  [1, 1.7320508075688776],  [0, 6.9282032302755105]]
    pos = [[0.5, 7.794228634059948], [9, 6.9282032302755105], [3, 0.0], [5.5, 0.8660254037844388], [0, 5.196152422706632], [5, 5.196152422706632], [9, 5.196152422706632], [0.5, 2.598076211353316], [7.5, 7.794228634059948], [4.5, 7.794228634059948], [0, 0.0], [9, 0.0], [9.5, 7.794228634059948], [9.5, 2.598076211353316]]
#    gene_list_top20 =['SPOP', 'TP53',  'FOXA1',  'CTNNB1',  'MED12',  'C16orf62',  'CLPTM1L',  'DPYSL2',  'NEIL1',  'SLC27A4',  'PITPNM2',  'PTEN', 'ATM',  'EMG1',  'ETV3',  'BRAF',  'NKX3-1',  'ZMYM3',  'OR4P4',  'SALL1']
    gene_list_top14 = ['RUNX1', 'PIK3CA', 'GATA3', 'FOXA1', 'SF3B1', 'PTEN', 'CBFB', 'CDH1', 'MAP2K4', 'MAP3K1', 'ERBB2', 'NCOR1', 'FAM86B2', 'CDKN1B']
    gene_list = gene_list_top14
#    features_selected_withprefix_GE = ['GE_' + feature for feature in gene_list]
#    features_selected_withprefix_DM = ['DM_' + feature for feature in gene_list]
#    features_selected_withprefix_CN = ['CNA_' + feature for feature in gene_list]
#    column_list = ['']
#    column_list = column_list.append(pd.Index(features_selected_withprefix_GE))
#    column_list = column_list.append(pd.Index(features_selected_withprefix_DM))
#    column_list = column_list.append(pd.Index(features_selected_withprefix_CN))

    pathGE = "C:\\PR_Proj_Thesis\\NEW_SOM_Approach\\BRCA\\Mergefile_top20_norm.csv"
    # Read file
    df = pd.read_csv(pathGE)

    df1 = df.loc[df['TUMOR_STAGE'] == 'Stage IIIA']
#    labels = df1['TUMOR_STAGE'].values
#
    df2 = df.loc[df['TUMOR_STAGE'] == 'Stage IIA']
#    labels2 = df2['TUMOR_STAGE'].values
#
    df3 = df.loc[df['TUMOR_STAGE'] == 'Stage IIB']
#    labels3 = df3['TUMOR_STAGE'].values

    df1.drop(['TUMOR_STAGE','PATIENT_ID'],axis =1,inplace = True)
    df2.drop(['TUMOR_STAGE','PATIENT_ID'],axis =1,inplace = True)
    df3.drop(['TUMOR_STAGE','PATIENT_ID'],axis =1,inplace = True)

    # applying scaling to make values between some range 0-1/-1-2 ,as need for Kohens SOM
    scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
    df1 = pd.DataFrame(scaler.fit_transform(df1), columns=df1.columns)
    df2 = pd.DataFrame(scaler.fit_transform(df2), columns=df2.columns)
    df3 = pd.DataFrame(scaler.fit_transform(df3), columns=df3.columns)

    col_num = 14
    #Create node list of numbers then create lables and dict
    node_list = [i for i in range(col_num)]
    new_lbl = [str(j) for j in range(col_num)]
    new_lbl_dict = dict(enumerate(new_lbl))

    # loop through all rows and get RGB values for each patient  ----- 3A
    row_count = df1.shape[0]

    for i in range(row_count): #loop through all rows
        nodecolor =[]
        for j in range(len(gene_list)): # loop through all nodes (i.e 14 nodes)

            r_prefix = 'GE_'+gene_list[j]
            g_prefix = 'DM_'+gene_list[j]
            b_prefix = 'CNA_'+gene_list[j]

            #Check if tht column exixts

            R =  round(df1[r_prefix][i],5) if (r_prefix in df1.columns) else 0.0
            G =  round(df1[g_prefix][i],5) if (g_prefix in df1.columns) else 0.0
            B =  round(df1[b_prefix][i],5) if (b_prefix in df1.columns) else 0.0

            nodecolor.append([R,G,B])

        G=nx.chvatal_graph()
        try:
            nx.draw_networkx_nodes(G,pos,
                               nodelist=node_list,
                               edgecolors = [0,0,0],
                               node_color = nodecolor,
                               node_size=500,
                               alpha=0.8)

            nx.draw_networkx_labels(G,pos,new_lbl_dict,font_size=10)

            plt.axis('on')
            filename = str(i)+'_Template.png'
            path = "./training/3A"
            plt.savefig(os.path.join(path,filename))#, bbox_inches='tight', dpi=72)
        except:
            print("row = ",i," node_color = ",nodecolor)

        print(("\r Preparing Images... "+str(int(i*100.0/row_count))+"%" ), end=' ')
    print("Done ---- 3A!!")
# loop through all rows and get RGB values for each patient  ----- 2A
    row_count = df2.shape[0]

    for i in range(row_count): #loop through all rows
        nodecolor =[]
        for j in range(len(gene_list)): # loop through all nodes (i.e 14 nodes)

            r_prefix = 'GE_'+gene_list[j]
            g_prefix = 'DM_'+gene_list[j]
            b_prefix = 'CNA_'+gene_list[j]

            #Check if tht column exixts

            R =  round(df2[r_prefix][i],5) if (r_prefix in df2.columns) else 0.0
            G =  round(df2[g_prefix][i],5) if (g_prefix in df2.columns) else 0.0
            B =  round(df2[b_prefix][i],5) if (b_prefix in df2.columns) else 0.0

            nodecolor.append([R,G,B])

        G=nx.chvatal_graph()
        try:
            nx.draw_networkx_nodes(G,pos,
                               nodelist=node_list,
                               edgecolors = [0,0,0],
                               node_color = nodecolor,
                               node_size=500,
                               alpha=0.8)

            nx.draw_networkx_labels(G,pos,new_lbl_dict,font_size=10)

            plt.axis('on')
            filename = str(i)+'_Template.png'
            path = "./training/2A"
            plt.savefig(os.path.join(path,filename))#, bbox_inches='tight', dpi=72)
        except:
            print("row = ",i," node_color = ",nodecolor)
        print(("\r Preparing Images... "+str(int(i*100.0/row_count))+"%" ), end=' ')
    print("Done ---- 2A!!")
# loop through all rows and get RGB values for each patient  ----- 2B
    row_count = df3.shape[0]

    for i in range(row_count): #loop through all rows
        nodecolor =[]
        for j in range(len(gene_list)): # loop through all nodes (i.e 14 nodes)

            r_prefix = 'GE_'+gene_list[j]
            g_prefix = 'DM_'+gene_list[j]
            b_prefix = 'CNA_'+gene_list[j]

            #Check if tht column exixts

            R =  round(df3[r_prefix][i],5) if (r_prefix in df3.columns) else 0.0
            G =  round(df3[g_prefix][i],5) if (g_prefix in df3.columns) else 0.0
            B =  round(df3[b_prefix][i],5) if (b_prefix in df3.columns) else 0.0

            nodecolor.append([R,G,B])

        G=nx.chvatal_graph()
        try:
            nx.draw_networkx_nodes(G,pos,
                               nodelist=node_list,
                               edgecolors = [0,0,0],
                               node_color = nodecolor,
                               node_size=500,
                               alpha=0.8)

            nx.draw_networkx_labels(G,pos,new_lbl_dict,font_size=10)

            plt.axis('on')
            filename = str(i)+'_Template.png'
            path = "./training/2B"
            plt.savefig(os.path.join(path,filename))#, bbox_inches='tight', dpi=72)
        except:
            print("row = ",i," node_color = ",nodecolor)
        print(("\r Preparing Images... "+str(int(i*100.0/row_count))+"%" ), end=' ')

print("Done ---- whole!!")

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 13 04:46:46 2019

"""

import shutil
import os
import random
def Movefiles(source,destination,num):

    # get list of files from Source(training) folder
    list_of_file = [os.path.abspath(os.path.join(source,x)) for x in os.listdir(source)]#os.path.abspath(x)
    #Randomly select files and put it in test folder
    for i in range(num):
        cut = random.choice(list_of_file)
        shutil.move(cut, destination)
        list_of_file = [os.path.abspath(os.path.join(source,x)) for x in os.listdir(source)]#os.path.abspath(x)
    print ('Done moving '+ str(num) +' files from source : ' ,source, ' to destination :',destination)

if __name__ == "__main__":
    num = 30
    src_2A = 'PacMAp\\training\\2A'
    dst_2A = 'PacMAp\\test\\2A'

    src_2B = 'PacMap\\training\\2B'
    dst_2B = 'PacMap\\test\\2B'

    src_3A = 'PacMap\\training\\3A'
    dst_3A = 'PacMap\\test\\3A'

    Movefiles(src_2A,dst_2A,num)
    Movefiles(src_2B,dst_2B,num)
    Movefiles(src_3A,dst_3A,num)

    print('*****************************DONE*************************************')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 22 06:51:30 2019
"""
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D,BatchNormalization,Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
#Imports for collecting metrics
import keras_metrics as km
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
#import tensorflow.keras as keras

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(BatchNormalization())
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2),strides = (2,2)))
classifier.add(Dropout(0.2))
# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2),strides = (2,2)))
classifier.add(Dropout(0.5))
### Adding a 3rd convolutional layer
#classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
#classifier.add(BatchNormalization())
#classifier.add(MaxPooling2D(pool_size = (2, 2),strides = (2,2)))
#classifier.add(Dropout(0.2))
## Adding a 4th convolutional layer
#classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
##classifier.add(BatchNormalization())
#classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(0.2))
# Step 3 - Flattening
classifier.add(Flatten())


# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.2))
#classifier.add(Dense(output_dim =1, activation = 'sigmoid'))# binary
classifier.add(Dense(output_dim =3, activation = 'softmax')) # catgorical
# SET METRICS

precision = km.categorical_precision()
recall = km.categorical_recall()
f1= km.categorical_f1_score()

# Compiling the CNN
#classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy',precision,recall,f1])
# # checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]




# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
#                                    rescale = 1./255,
#                                   shear_range = 0.2,
#                                   zoom_range = 0.2,
#                                   horizontal_flip = True,
#                                   validation_split = 0.7)

test_datagen = ImageDataGenerator()#rescale = 1./255)

seed =7
training_set = train_datagen.flow_from_directory('training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 shuffle =True,
                                                 seed =seed)#     seed =seed)#,save_to_dir = 'generatedimages') #categorical,binary

test_set = test_datagen.flow_from_directory('test',
                                            target_size = (64, 64),
                                            batch_size =32,
                                            class_mode = 'categorical',
                                            shuffle =False)#,   seed =seed)#categorical,binary
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    classifier.fit_generator(training_set,
                         samples_per_epoch = 150,
                         nb_epoch =30,
                         validation_data = test_set,
                         nb_val_samples = 60,
                         shuffle =True,
                         callbacks=callbacks_list,
                         verbose = 2)
